### VGG16

In [3]:
tf.__version__

'1.13.1'

In [2]:
###  VGG16 
import tensorflow as tf

class VGG16(object):
    def __init__(self):
        self.learning_rate = 0.00003

    def __conv2d(self, name, inputs, filters, kernel_size=(3, 3), activation='relu', padding='same'):

        return tf.layers.conv2d(inputs, filters, kernel_size,
                                padding=padding,
                                activation=tf.nn.relu,
                                name=name)
    def __conv_block(self, name, inputs, filters, kernel_size=(3, 3), activation='relu', padding='same'):

        x = inputs
        i = 0
        for filter in filters:
            x = self.__conv2d('{}_conv{}'.format(name, i), x, filter, kernel_size, activation)
            i = i + 1
        return tf.layers.max_pooling2d(inputs=x, pool_size=[2, 2], strides=2, padding=padding,
                                       data_format='channels_last', name=name + '_pool')

    def build(self):
        self.phase_train_placeholder = tf.placeholder(tf.bool, name='phase_train')
        self.img_batch = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name='input_img')
        self.label_batch = tf.placeholder(tf.float32, shape=[None, 4], name='input_label')
        x = self.__conv_block('conv_block1', self.img_batch, [64, 64])
        x = self.__conv_block('conv_block2', x, [128, 128])
        x = self.__conv_block('conv_block3', x, [256, 256, 256])
        x = self.__conv_block('conv_block4', x, [512, 512, 512])
        x = self.__conv_block('conv_block5', x, [512, 512, 512])
        x = tf.layers.flatten(x)
        x = tf.layers.dense(x, units=2048, activation=tf.nn.relu, name='fc1')
        x = tf.cond(self.phase_train_placeholder, lambda: tf.layers.dropout(x, 0.5, training=True),
                    lambda: tf.layers.dropout(x, 0.5, training=False))
        x = tf.layers.dense(x, units=4096, activation=tf.nn.relu,name='fc2')
        x = tf.cond(self.phase_train_placeholder, lambda: tf.layers.dropout(x, 0.5, training=True),
                    lambda: tf.layers.dropout(x, 0.5, training=False))
        self.logits = tf.layers.dense(x, units=4, name="logits")
        self.predictions = tf.nn.softmax(self.logits, name="predictions")
        self.loss = -tf.reduce_mean(self.label_batch * tf.log(self.predictions), name='loss')
        correct_prediction = tf.equal(tf.argmax(self.label_batch, 1), tf.argmax(self.predictions, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        self.opt = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)

### 数据读取/图片裁剪

In [3]:
### 数据读取与预处理
import scipy
from scipy import io
import glob, os
import numpy as np
import cv2
from numpy import array

sub_types = ['B-cell', 'T-cell', 'Monocyte', 'Granualocyte']
sub_labels = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
traval_indiv = ['HS1', 'HS2', 'HS3', 'HS5', 'HS6']
test = ['WBC_QPI_022219']

def get_tif(file_path, img_size):

    phase_img = cv2.imread(file_path, 1)
    phase_img = cv2.resize(phase_img, (img_size, img_size))
    return phase_img

def load_data_path(dirpath, traval_indiv):

    img_width, img_height = 224, 224
    nb_classes = 4
    data_set = []
    target_set = []

    for dd in traval_indiv:
        file_name = os.path.join(dirpath, dd)
        for i, cell_type in enumerate(sub_types):
            sub_dirs = file_name + '\\' + cell_type + '\\'
            os.chdir(sub_dirs)
            all_cell = glob.glob('*.tif')
            for file in all_cell:
                file_path = sub_dirs + file
                data_set.append(file_path)
                target_set.append(sub_labels[i])

    return data_set, target_set

def load_train(sdir,traval_indiv):

    data_sets = []
    target_sets = []
    for d in traval_indiv:
        print(d)
        dpath = os.path.join(sdir, d)
        data_set, target_set = load_data(dpath,sub_types)
        data_sets.extend(data_set.tolist())
        target_sets.extend(target_set)
    return data_sets, target_sets

if __name__ == "__main__":
    train_dir = 'E:\PythonProject\cell_pic'
    #load_train(train_dir, traval_indiv)

## 训练

1. 利用 HS1,HS2,HS3,HS5,HS6 的数据进行训练，WBC_QPI_022219的数据进行验证

2. epoch = 10, batch_size = 14, learning = 0.00003，每200个batch_size修改learning_rate = 0.95 * learning_rate

3. 每50个batch_size保存一次模型文件并使用WBC_QPI_022219的数据集验证一次模型的准确度

In [ ]:
import tensorflow as tf
import os
import numpy as np
import logging
import math
import random
traval_indiv = ['HS1', 'HS2', 'HS3', 'HS5', 'HS6']
test_indiv = ['WBC_QPI_022219']

def train():
    img_size = 224
    train_dir = 'E:\PythonProject\cell_pic'
    imgpaths, labels = load_data_path(train_dir, traval_indiv)
    train_num = len(imgpaths)
    test_imgpaths, test_labels = load_data_path(train_dir, test_indiv)
    from datetime import datetime
    model_dir = 'E:\PythonProject\model_save_file'
    logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(levelname)s %(message)s',datefmt='%a, %d %b %Y %H:%M:%S', filename=model_dir + '\log.txt', filemode='w')
    logger = logging.getLogger(__name__)
    subdir = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
    dst = os.path.join(model_dir, subdir)
    os.mkdir(dst)
    model = VGG16()
    
    # 重新设置图
    tf.reset_default_graph()
    
    with tf.Session() as sess:
        model.build()
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep=20)
        saver.restore(sess, 'E:\\mygit\\PythonLearn\\computer_vision\\model_save_file\\20190802-142456/model-93.0-loss0.090.ckpt-1800')
        bacth_size = 14
        epoch_size = math.floor(train_num / bacth_size)
        indexs = [x for x in range(train_num)]

        for j in range(10):
            random.shuffle(indexs)
            imgpaths = np.array(imgpaths)[indexs]
            labels = np.array(labels)[indexs]
            tloss = 0
            for i in range(int(epoch_size)):
                imgpath_batch = imgpaths[i * bacth_size:bacth_size * (i + 1)]
                img_batch = [get_tif(imgpath, img_size) for imgpath in imgpath_batch]
                img_batch = np.array(img_batch).reshape((bacth_size, img_size, img_size, 3))
                label_batch = labels[i * bacth_size:bacth_size * (i + 1)]
                label_batch = np.array(label_batch).reshape((bacth_size, 4))
                accuracy, loss, _ = sess.run([model.accuracy, model.loss, model.opt],
                                             feed_dict={model.phase_train_placeholder: True, model.img_batch: img_batch,
                                                        model.label_batch: label_batch})

                print('ecoph {}-{}/{},loss:{},accuracy:{}'.format(j, i, epoch_size, loss, accuracy))

                tloss = tloss + loss
                step = j * epoch_size + i
                if step % 50 == 0 and step != 0:
                    tloss = tloss / 50
                    val_accuracy, val_loss = validate(sess, model, test_imgpaths, test_labels, img_size)
                    info = 'ecoph{}-{}-val_accuracy:{}-val_loss:{}-train_loss:{}'.format(j, step, val_accuracy,
                                                                                         val_loss, tloss)
                    logger.info(info)
                    print(info)
                    checkpoint_path = os.path.join(dst,
                                                   'model-{:.1f}-loss{:.3f}.ckpt'.format(val_accuracy * 100, tloss))

                    saver.save(sess, checkpoint_path, global_step=step, write_meta_graph=True)

                if step % 200 == 0 and step != 0:
                    model.learning_rate = model.learning_rate * 0.95
                    print('learning_rate:{}'.format(model.learning_rate))
                    logger.info('learning_rate:{}'.format(model.learning_rate))


def validate(sess, model, imgpaths, labels, img_size):
    val_num = len(labels)
    bacth_size = 10
    epoch_size = int(math.floor(val_num / bacth_size))
    total_accu = 0
    total_loss = 0
    indexs = [x for x in range(val_num)]
    random.shuffle(indexs)
    imgpaths = np.array(imgpaths)[indexs]
    labels = np.array(labels)[indexs]
    for i in range(epoch_size):
        imgpath_batch = imgpaths[i * bacth_size:bacth_size * (i + 1)]
        try:
            img_batch = [get_tif(imgpath, img_size) for imgpath in imgpath_batch]
            img_batch = np.array(img_batch).reshape((bacth_size, img_size, img_size, 3))
        except Exception as e:
            print(e)
            continue
        label_batch = labels[i * bacth_size:bacth_size * (i + 1)]
        label_batch = np.array(label_batch).reshape((bacth_size, 4))
        accuracy, loss = sess.run([model.accuracy, model.loss],
                                  feed_dict={model.phase_train_placeholder: False, model.img_batch: img_batch,
                                             model.label_batch: label_batch})
        total_accu += accuracy
        total_loss += loss

    return total_accu / epoch_size, total_loss / epoch_size

train()

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from E:\mygit\PythonLearn\computer_vision\model_save_file\20190802-142456/model-93.0-loss0.090.ckpt-1800
ecoph 0-0/183,loss:0.03779304772615433,accuracy:1.0
ecoph 0-1/183,loss:0.04926013946533203,accuracy:0.9285714030265808
ecoph 0-2/183,loss:0.08621936291456223,accuracy:0.7857142686843872
ecoph 0-3/183,loss:0.042248401790857315,accuracy:0.9285714030265808
ecoph 0-4/183,loss:0.0

### 计算WBC_QPI_022219数据集三种细胞F1值

1. 利用训练过程中在验证集上达到准确度最高的模型为最终模型，对验证集数据进行F1计算
 
2. 分别计算每种细胞的F1值

3. F1值计算结果如下:

|Model|Test person|Age|Gender|Ethnicity|Weight|Height|Dataset|B|T|M|G|
|:----|:----|:---|:----- |-----|--|--|--|--|---|---|---|
|ResNet(baseline)|5|37|male|African american|87kg|179cm|WBC_QPI_022219|0/0%|251/93%|202/85%|192/83%|
|vgg16|5|37|male|African american|87kg|179cm|WBC_QPI_022219|0/0%|251/93.5%|202/91.6%|192/95.4%|

In [ ]:
import tensorflow as tf
import cv2
import numpy as np

class TestInterface():
    
    def __init__(self, sess, hat_model_path):
        self.model = VGG16()
        self.model.build()
        self.sess = sess
        self.img_size = 224
        saver = tf.train.Saver()
        saver.restore(self.sess, hat_model_path)

    def validate(self, testdir):
        imgsize = self.img_size
        test_indiv = ['WBC_QPI_022219']
        imgpaths, labels = load_data_path(testdir, test_indiv)
        sess = self.sess
        model = self.model
        imgnum = len(labels)
        pre_labels = []
        tp = [0, 0, 0, 0]
        for i in range(imgnum):
            print(i, '/', imgnum)
            img = cv2.imread(imgpaths[i], 1)
            
            # picture resize
            if img.shape[0] != imgsize or img.shape[1] != imgsize:
                img = cv2.resize(img, (imgsize, imgsize))
                
            img_batch = np.array([img]).reshape((1, imgsize, imgsize, 3))
            label_batch = [labels[i]]
            predictions = sess.run(model.predictions,
                                   feed_dict={model.phase_train_placeholder: False, model.img_batch: img_batch})
            pclassname = int(np.argmax(predictions, 1))
            isright = np.equal(np.argmax(label_batch, 1), np.argmax(predictions, 1))
            x = [0, 0, 0, 0]
            x[pclassname] = 1
            pre_labels.append(x)
            if isright[0]:
                tp[pclassname] += 1

        tp_fn = np.sum(np.array(labels), axis=0)
        tp_fp = np.sum(np.array(pre_labels), axis=0)
        precision = tp / (tp_fp + 1e-6)
        recall = tp / (tp_fn + 1e-6)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
        print(precision, recall, f1)

import logging

if __name__ == '__main__':
    hat_model_path = 'E:\PythonProject/model_save_file\20190802-142456/model-93.0-loss0.090.ckpt-1800'
    
    with tf.Session() as sess:
        test = TestInterface(sess, hat_model_path)
        test.validate('E:\PythonProject\cell_pic')

In [3]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# 新建session with log_device_placement并设置为True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# 运行这个 op.
print (sess.run(c))

[[22. 28.]
 [49. 64.]]
